## Multivariate Multi-step Time Series Forecasting using Stacked LSTM sequence to sequence Autoencoder in Tensorflow 2.0 / Keras

This article will see how to create a stacked sequence to sequence the LSTM model for time series forecasting in Keras/ TF 2.0. 

In Sequence to Sequence Learning, an RNN model is trained to map an input sequence to an output sequence. The input and output need not necessarily be of the same length. The seq2seq model contains two RNNs, e.g., LSTMs. They can be treated as an encoder and decoder. The encoder part converts the given input sequence to a fixed-length vector, which acts as a summary of the input sequence.

This fixed-length vector is called the context vector. The context vector is given as input to the decoder and the final encoder state as an initial decoder state to predict the output sequence. Sequence to Sequence learning is used in language translation, speech recognition, time series
forecasting, etc.

We will use the sequence to sequence learning for time series forecasting. We can use this architecture to easily make a multistep forecast. we will add two layers, a repeat vector layer and time distributed dense layer in the architecture.

A repeat vector layer is used to repeat the context vector we get from the encoder to pass it as an input to the decoder. We will repeat it for n-steps ( n is the no of future steps you want to forecast). The output received from the decoder with respect to each time step is mixed. The time distributed densely will apply a fully connected dense layer on each time step and separates the output for each timestep. The time distributed densely is a wrapper that allows applying a layer to every temporal slice of an input.

We will stack additional layers on the encoder part and the decoder part of the sequence to sequence model. By stacking LSTM’s, it may increase the ability of our model to understand more complex representation of our time-series data in hidden layers, by capturing information at different levels.

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import tensorflow as tf
from pyFTS.models.nonstationary import nsfts
from pyFTS.benchmarks import Measures
from pyFTS.benchmarks import Measures
import matplotlib.pyplot as plt
from pyFTS.common import Util
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score
import datetime
import statistics
import math
import os
import sys
import statistics
sys.path.append("/home/hugo/projetos-doutorado/mimo_emb_fts/src/")

from embfts.util.DataSetUtil import DataSetUtil
from embfts.util.StatisticsUtil import StatisticsUtil

In [2]:
data_set_util = DataSetUtil()
statistics_util = StatisticsUtil()

In [3]:
def cal_nrmse(rmse, y):
    x = max(y)-min(y)
    return (rmse/x)

In [4]:
df = pd.read_csv('/home/hugo/projetos-doutorado/mimo_emb_fts/data/energydata_complete.csv', sep=',')
data = df.drop(labels=['date','rv1','rv2'], axis=1)
data = data_set_util.clean_dataset(data)
data = data_set_util.series_to_supervised_mimo(data, 1, 1)
data.head()


,Appliances(t-1),lights(t-1),T1(t-1),RH_1(t-1),T2(t-1),RH_2(t-1),T3(t-1),RH_3(t-1),T4(t-1),RH_4(t-1),...,T8(t),RH_8(t),T9(t),RH_9(t),T_out(t),Press_mm_hg(t),RH_out(t),Windspeed(t),Visibility(t),Tdewpoint(t)
1,60.0,30.0,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,...,18.2,48.863333,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2
2,60.0,30.0,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,...,18.2,48.730000,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1
3,50.0,30.0,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,...,18.1,48.590000,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0
4,50.0,40.0,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,...,18.1,48.590000,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9
5,60.0,40.0,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,...,18.1,48.590000,17.000000,45.29,6.016667,734.0,92.0,5.333333,43.833333,4.8


## Model Architecture

E1D1 ==> Sequence to Sequence Model with one encoder layer and one decoder layer.

In [5]:
def create_lstm_model_d1(neurons, n_past, n_future, n_features):
    encoder_inputs = tf.keras.layers.Input(shape=(n_past, n_features))
    encoder_l1 = tf.keras.layers.LSTM(neurons, return_state=True)
    encoder_outputs1 = encoder_l1(encoder_inputs)

    encoder_states1 = encoder_outputs1[1:]
    decoder_inputs = tf.keras.layers.RepeatVector(n_future)(encoder_outputs1[0])

    decoder_l1 = tf.keras.layers.LSTM(neurons, return_sequences=True)(decoder_inputs,initial_state = encoder_states1)
    decoder_outputs1 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_features))(decoder_l1)

    model_e1d1 = tf.keras.models.Model(encoder_inputs,decoder_outputs1)

    return model_e1d1

E2D2 ==> Sequence to Sequence Model with two encoder layers and two decoder layers.

In [6]:
def create_lstm_model_d2(neurons, n_past, n_future, n_features):
    encoder_inputs = tf.keras.layers.Input(shape=(n_past, n_features))
    encoder_l1 = tf.keras.layers.LSTM(neurons,return_sequences = True, return_state=True)
    encoder_outputs1 = encoder_l1(encoder_inputs)
    encoder_states1 = encoder_outputs1[1:]
    encoder_l2 = tf.keras.layers.LSTM(neurons, return_state=True)
    encoder_outputs2 = encoder_l2(encoder_outputs1[0])
    encoder_states2 = encoder_outputs2[1:]
    #
    decoder_inputs = tf.keras.layers.RepeatVector(n_future)(encoder_outputs2[0])
    #
    decoder_l1 = tf.keras.layers.LSTM(neurons, return_sequences=True)(decoder_inputs,initial_state = encoder_states1)
    decoder_l2 = tf.keras.layers.LSTM(neurons, return_sequences=True)(decoder_l1,initial_state = encoder_states2)
    decoder_outputs2 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_features))(decoder_l2)
    #
    model_e2d2 = tf.keras.models.Model(encoder_inputs,decoder_outputs2)
    #
    return model_e2d2
    

In [7]:
def compile_lstm_model(epochs, model, X_train,y_train, X_test,y_test, batch_size, verbose):
    reduce_lr = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.90 ** x)
    model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.Huber())
    history_model = model.fit(X_train,y_train,epochs=epochs,validation_data=(X_test,y_test),batch_size=batch_size,verbose=verbose,callbacks=[reduce_lr])
    return model, history_model

In [8]:
def plot_history_model(history_model):
    plt.plot(history_model.history['loss'])
    plt.plot(history_model.history['val_loss'])
    plt.title("Model Loss")
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend(['Train', 'Valid'])
    plt.show()

In [9]:
def sliding_window(data,n_windows,train_size,epochs,batch_size,verbose,str_model,neurons,n_past,n_future,n_features):

    result = {
         "window": [],
         "rmse": [],
         "mape": [],
         "mae": [],
         "r2": [],
         "smape": [],
         "nrmse": [],
         "variable":[]
    }
    
    final_result = {
         "window": [],
         "rmse": [],
         "mape": [],
         "mae": [],
         "r2": [],
         "smape": [],
         "nrmse": [],
         "variable":[]
    }

    tam = len(data)
    windows_length = math.floor(tam / n_windows)
    for ct, ttrain, ttest in Util.sliding_window(data, windows_length, train_size, inc=1):
        if len(ttest) > 0:
            
            print('-' * 20)
            print(f'training window {(ct)}')
            
            scaler = StandardScaler()

            Xtrain = scaler.fit_transform(ttrain.loc[:,'Appliances(t-1)':'Tdewpoint(t-1)'])
            ytrain = scaler.fit_transform(ttrain.loc[:,'Appliances(t)':'Tdewpoint(t)'])
            Xtest = scaler.transform(ttest.loc[:,'Appliances(t-1)':'Tdewpoint(t-1)'])
            ytest = scaler.transform(ttest.loc[:,'Appliances(t)':'Tdewpoint(t)'])
            
#             Xval = Xtest[:int(len(Xtest)*0.25)]
#             yval = ytest[:int(len(ytest)*0.25)]

            Xval = Xtrain[:int(len(Xtrain)*0.15)]
            yval = ytrain[:int(len(ytrain)*0.15)]
            
            X_train = Xtrain.reshape(Xtrain.shape[0], 1, Xtrain.shape[1])
            y_train = ytrain.reshape(ytrain.shape[0], 1, ytrain.shape[1])
            X_test = Xtest.reshape(Xtest.shape[0], 1, Xtest.shape[1])
            y_test = ytest.reshape(ytest.shape[0], 1, ytest.shape[1])
            X_val = Xval.reshape(Xval.shape[0], 1, Xval.shape[1])
            y_val = yval.reshape(yval.shape[0], 1, yval.shape[1])
            
            if str_model == 'd1':
                model = create_lstm_model_d1(neurons, n_past, n_future, n_features)
                model,history_model = compile_lstm_model(epochs,model,X_train,y_train,X_val,y_val,batch_size,verbose)
            else:
                model = create_lstm_model_d2(neurons, n_past, n_future, n_features)
                model,history_model = compile_lstm_model(epochs,model,X_train,y_train,X_val,y_val,batch_size,verbose)
             
            #plot_history_model(history_model)
            
            df_forecats_columns = ['Appliances(t)', 'lights(t)', 'T1(t)', 'RH_1(t)', 'T2(t)', 'RH_2(t)',
                                   'T3(t)', 'RH_3(t)', 'T4(t)', 'RH_4(t)', 'T5(t)', 'RH_5(t)', 'T6(t)',
                                   'RH_6(t)', 'T7(t)', 'RH_7(t)', 'T8(t)', 'RH_8(t)', 'T9(t)', 'RH_9(t)',
                                   'T_out(t)', 'Press_mm_hg(t)', 'RH_out(t)', 'Windspeed(t)',
                                   'Visibility(t)', 'Tdewpoint(t)']
            columns = list(df_forecats_columns)
            
            prediction = model.predict(X_test)
            prediction = prediction.reshape(prediction.shape[0], prediction.shape[2])
            prediction = scaler.inverse_transform(prediction)            
            df_forecast = pd.DataFrame(prediction,columns=columns)
            
            #ytest_metric = scaler.inverse_transform(y_test.reshape(y_test.shape[0], y_test.shape[2]))
            ytest_metric = ttest.loc[:,'Appliances(t)':'Tdewpoint(t)'].values
            df_original = pd.DataFrame(ytest_metric,columns=columns)
            
            for col in columns:  
                original = df_original[col].values
                forecast = df_forecast[col].values
#                 original = original[:len(original)-1]
#                 forecast = forecast[1:]
                
                mae = round(mean_absolute_error(original,forecast),3)
                r2 = round(r2_score(original,forecast),3)
                #rmse = mean_squared_error(original,forecast,squared=False)
                rmse = round(Measures.rmse(original,forecast),3)
                mape = round(Measures.mape(original,forecast),3)
                nrmse = round(cal_nrmse(rmse, original),3)
                smape = round(Measures.smape(original,forecast),3)

                result["rmse"].append(rmse)
                result["nrmse"].append(nrmse)
                result["mape"].append(mape)
                result["mae"].append(mae)
                result["r2"].append(r2)
                result["smape"].append(smape)
                result["window"].append(ct)
                result["variable"].append(col)
                
#                 fig, ax = plt.subplots(nrows=1, ncols=1, figsize=[15, 3])
#                 ax.plot(original, label='Original')
#                 ax.plot(forecast, label='Forecast')
#                 handles, labels = ax.get_legend_handles_labels()
#                 lgd = ax.legend(handles, labels, loc=2, bbox_to_anchor=(1, 1))
#                 plt.show()
        
    measures = pd.DataFrame(result)
    return measures

In [10]:
n_windows = 30
train_size = 0.75 
epochs = 25
batch_size = 32
verbose = 0
neurons = 200 
n_past = 1 
n_future = 1 
n_features = 26
str_model = 'd1'

model_d1_result = sliding_window(data,n_windows,train_size,epochs,batch_size,verbose,str_model,neurons,n_past,n_future,n_features)

--------------------
training window 0


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100


--------------------
training window 657


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100


--------------------
training window 1314


...

/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100


--------------------
training window 18396


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100


--------------------
training window 19053


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100


In [11]:
df_forecats_columns = ['Appliances(t)', 'lights(t)', 'T1(t)', 'RH_1(t)', 'T2(t)', 'RH_2(t)',
                                   'T3(t)', 'RH_3(t)', 'T4(t)', 'RH_4(t)', 'T5(t)', 'RH_5(t)', 'T6(t)',
                                   'RH_6(t)', 'T7(t)', 'RH_7(t)', 'T8(t)', 'RH_8(t)', 'T9(t)', 'RH_9(t)',
                                   'T_out(t)', 'Press_mm_hg(t)', 'RH_out(t)', 'Windspeed(t)',
                                   'Visibility(t)', 'Tdewpoint(t)']

columns = list(df_forecats_columns)

final_result = {
    "variable": [],
    "rmse": [],
    "mae": [],
    "mape": [],
    "r2": [],
    "smape": [],
    "nrmse": [],
    "rmse_std": [],
    "mae_std": [],
    "mape_std": [],
    "r2_std": [],
    "smape_std": [],
    "nrmse_std": []
}

model_d1_result.to_csv (r'all_windows_stacked_lstm_d1_uci_appliances.csv', index = False, header=True)

measures = model_d1_result
var = measures.groupby("variable")

for col in columns:
    
    var_agr = var.get_group(col)
           
    rmse = round(statistics.mean(var_agr.loc[:,'rmse']),3)
    mape = round(statistics.mean(var_agr.loc[:,'mape']),3)
    mae = round(statistics.mean(var_agr.loc[:,'mae']),3)
    r2 = round(statistics.mean(var_agr.loc[:,'r2']),3)
    smape = round(statistics.mean(var_agr.loc[:,'smape']),3)
    nrmse = round(statistics.mean(var_agr.loc[:,'nrmse']),3)
    
    rmse_std = round(statistics.stdev(var_agr.loc[:,'rmse']),3)
    mape_std = round(statistics.stdev(var_agr.loc[:,'mape']),3)
    mae_std = round(statistics.stdev(var_agr.loc[:,'mae']),3)
    r2_std = round(statistics.stdev(var_agr.loc[:,'r2']),3)
    smape_std = round(statistics.stdev(var_agr.loc[:,'smape']),3)
    nrmse_std = round(statistics.stdev(var_agr.loc[:,'nrmse']),3)

    final_result["variable"].append(col)
    final_result["rmse"].append(rmse)
    final_result["mape"].append(mape)
    final_result["mae"].append(mae)
    final_result["r2"].append(r2)
    final_result["smape"].append(smape)
    final_result["nrmse"].append(nrmse)
    
    final_result["rmse_std"].append(rmse_std)
    final_result["mape_std"].append(mape_std)
    final_result["mae_std"].append(mae_std)
    final_result["r2_std"].append(r2_std)
    final_result["smape_std"].append(smape_std)
    final_result["nrmse_std"].append(nrmse_std)
        
    #print(f'Results: {(col,rmse,mae,r2)}')
        
        
final_measures = round(pd.DataFrame(final_result),3)

In [12]:
final_measures

,variable,rmse,mae,mape,r2,smape,nrmse,rmse_std,mae_std,mape_std,r2_std,smape_std,nrmse_std
0,Appliances(t),79.553,47.939,52.590,-3.630000e-01,21.135,0.174,32.477,19.496,28.980,2.205000e+00,7.038,0.111
1,lights(t),6.600,4.384,inf,-7.510000e-01,88.053,inf,2.982,2.420,NaN,3.812000e+00,10.780,NaN
2,T1(t),0.418,0.353,1.652,6.200000e-02,0.827,0.224,0.262,0.234,1.150,2.072000e+00,0.586,0.157
3,RH_1(t),1.226,0.935,2.326,2.390000e-01,1.163,0.157,0.682,0.608,1.482,1.049000e+00,0.749,0.081
4,T2(t),0.654,0.542,2.593,4.250000e-01,1.297,0.186,0.390,0.312,1.324,9.210000e-01,0.669,0.104
5,RH_2(t),1.472,1.208,3.021,-1.400000e-02,1.500,0.209,1.074,0.959,2.109,2.045000e+00,1.074,0.172
6,T3(t),0.552,0.459,2.035,-8.880000e-01,1.028,0.298,0.351,0.313,1.407,1.746000e+00,0.729,0.146
7,RH_3(t),0.994,0.821,2.099,-6.400000e-02,1.040,0.227,0.616,0.540,1.340,9.610000e-01,0.655,0.125
8,T4(t),0.694,0.586,2.893,-1.080000e+00,1.464,0.347,0.481,0.418,2.133,2.352000e+00,1.116,0.172
9,RH_4(t),1.090,0.898,2.293,-3.290000e-01,1.142,0.240,0.587,0.535,1.321,1.898000e+00,0.655,0.132


In [14]:
final_measures.to_csv (r'stacked_lstm_d1_uci_appliances_with_std.csv', index = False, header=True)

In [15]:
n_windows = 30
train_size = 0.75 
epochs = 25
batch_size = 32
verbose = 0
neurons = 200 
n_past = 1 
n_future = 1 
n_features = 26
str_model = 'd2'

model_d2_result = sliding_window(data,n_windows,train_size,epochs,batch_size,verbose,str_model,neurons,n_past,n_future,n_features)

--------------------
training window 0


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100


--------------------
training window 657


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100


--------------------
training window 1314


...

/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100


--------------------
training window 18396


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100


--------------------
training window 19053


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100


In [19]:
df_forecats_columns = ['Appliances(t)', 'lights(t)', 'T1(t)', 'RH_1(t)', 'T2(t)', 'RH_2(t)',
                                   'T3(t)', 'RH_3(t)', 'T4(t)', 'RH_4(t)', 'T5(t)', 'RH_5(t)', 'T6(t)',
                                   'RH_6(t)', 'T7(t)', 'RH_7(t)', 'T8(t)', 'RH_8(t)', 'T9(t)', 'RH_9(t)',
                                   'T_out(t)', 'Press_mm_hg(t)', 'RH_out(t)', 'Windspeed(t)',
                                   'Visibility(t)', 'Tdewpoint(t)']

columns = list(df_forecats_columns)

final_result = {
    "variable": [],
    "rmse": [],
    "mae": [],
    "mape": [],
    "r2": [],
    "smape": [],
    "nrmse": [],
    "rmse_std": [],
    "mae_std": [],
    "mape_std": [],
    "r2_std": [],
    "smape_std": [],
    "nrmse_std": []
}

model_d2_result.to_csv (r'all_windows_stacked_lstm_d2_uci_appliances.csv', index = False, header=True)

measures = model_d2_result
var = measures.groupby("variable")

for col in columns:
    
    var_agr = var.get_group(col)
           
    rmse = round(statistics.mean(var_agr.loc[:,'rmse']),3)
    mape = round(statistics.mean(var_agr.loc[:,'mape']),3)
    mae = round(statistics.mean(var_agr.loc[:,'mae']),3)
    r2 = round(statistics.mean(var_agr.loc[:,'r2']),3)
    smape = round(statistics.mean(var_agr.loc[:,'smape']),3)
    nrmse = round(statistics.mean(var_agr.loc[:,'nrmse']),3)
    
    rmse_std = round(statistics.stdev(var_agr.loc[:,'rmse']),3)
    mape_std = round(statistics.stdev(var_agr.loc[:,'mape']),3)
    mae_std = round(statistics.stdev(var_agr.loc[:,'mae']),3)
    r2_std = round(statistics.stdev(var_agr.loc[:,'r2']),3)
    smape_std = round(statistics.stdev(var_agr.loc[:,'smape']),3)
    nrmse_std = round(statistics.stdev(var_agr.loc[:,'nrmse']),3)

    final_result["variable"].append(col)
    final_result["rmse"].append(rmse)
    final_result["mape"].append(mape)
    final_result["mae"].append(mae)
    final_result["r2"].append(r2)
    final_result["smape"].append(smape)
    final_result["nrmse"].append(nrmse)
    
    final_result["rmse_std"].append(rmse_std)
    final_result["mape_std"].append(mape_std)
    final_result["mae_std"].append(mae_std)
    final_result["r2_std"].append(r2_std)
    final_result["smape_std"].append(smape_std)
    final_result["nrmse_std"].append(nrmse_std)
        
    #print(f'Results: {(col,rmse,mae,r2)}')
        
        
final_measures_d2 = round(pd.DataFrame(final_result),3)

In [20]:
final_measures_d2

,variable,rmse,mae,mape,r2,smape,nrmse,rmse_std,mae_std,mape_std,r2_std,smape_std,nrmse_std
0,Appliances(t),87.311,52.930,57.418,-5.440000e-01,22.990,0.189,36.595,21.162,30.925,2.291000e+00,6.160,0.111
1,lights(t),7.046,4.589,inf,-4.930000e-01,89.936,inf,3.438,2.749,NaN,2.050000e+00,9.684,NaN
2,T1(t),0.459,0.389,1.824,-3.000000e-02,0.912,0.244,0.270,0.240,1.201,2.073000e+00,0.608,0.159
3,RH_1(t),1.461,1.131,2.796,-4.500000e-02,1.395,0.191,0.714,0.636,1.546,9.660000e-01,0.768,0.098
4,T2(t),0.746,0.618,2.957,2.370000e-01,1.478,0.213,0.436,0.363,1.549,1.211000e+00,0.785,0.122
5,RH_2(t),1.626,1.330,3.322,-1.970000e-01,1.654,0.234,1.071,0.976,2.125,1.999000e+00,1.092,0.174
6,T3(t),0.607,0.502,2.225,-1.203000e+00,1.126,0.322,0.375,0.335,1.513,2.036000e+00,0.787,0.147
7,RH_3(t),1.094,0.907,2.310,-2.540000e-01,1.144,0.248,0.651,0.563,1.384,1.015000e+00,0.673,0.126
8,T4(t),0.783,0.669,3.310,-1.694000e+00,1.675,0.388,0.525,0.459,2.368,2.937000e+00,1.236,0.193
9,RH_4(t),1.204,0.994,2.545,-4.400000e-01,1.265,0.269,0.518,0.452,1.152,1.330000e+00,0.570,0.126


In [21]:
final_measures_d2.to_csv (r'stacked_lstm_d2_uci_appliances_with_std.csv', index = False, header=True)